# Label_Prepocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os
import IPython
import glob
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score


from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.metrics import precision_score, recall_score, f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.layers import Embedding

In [ ]:
mfcc_csv=pd.read_csv("/content/drive/MyDrive/mini project/MFCC2.csv")

In [ ]:
label_list=mfcc_csv["label_list"]

In [ ]:
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(label_list)
print(label_encoded)

[12 12 12 ...  0  0  0]


In [ ]:
label_encoded = label_encoded[:, np.newaxis]


In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded)
one_hot_encoded

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Load the CSV file and extract the label_list
mfcc_csv = pd.read_csv("/content/drive/MyDrive/mini project/MFCC2.csv")
label_list = mfcc_csv["label_list"]

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(label_list)
label_encoded = label_encoded[:, np.newaxis]

# One-hot encode the labels using OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded)

# Save the label_encoder and one_hot_encoder objects
np.save('/content/drive/MyDrive/label_encoder_classes2.npy', label_encoder.classes_)
np.save('/content/drive/MyDrive/one_hot_encoder_matrix2.npy', one_hot_encoder.get_feature_names_out(['label_encoded']))

print("Label encoder classes and one-hot encoder matrix saved successfully.")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Label encoder classes and one-hot encoder matrix saved successfully.


# Model building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.models import Sequential

In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/mini project/MFCC2.csv")

In [ ]:
label_list=df["label_list"]

In [ ]:
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(label_list)
print(label_encoded)

[12 12 12 ...  0  0  0]


In [ ]:
label_encoded = label_encoded[:, np.newaxis]


In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded)
one_hot_encoded

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X = df.drop(['label_list'], axis=1)

In [ ]:
X

,0,1,2,3,4,5,6,7,8,9,10,11
0,-659.21630,18.278786,-12.107536,44.013153,-13.154768,5.019222,-5.396541,-3.083084,-9.133388,-5.569468,-11.617992,11.489901
1,-533.87380,33.250694,-24.455145,43.151917,-25.093418,-10.435495,-19.112411,-15.169186,-7.704080,-8.938352,-1.819665,16.766113
2,-468.43643,11.820848,-19.447367,33.744300,-24.006926,-10.317425,-12.795439,-13.549147,-10.402909,-10.476334,-5.665104,14.598053
3,-594.96625,10.194246,-8.782218,40.609173,-10.718865,0.291726,-3.624802,-6.048359,-7.304923,-6.239582,-8.779635,13.586450
4,-635.07110,58.709858,-32.574550,46.426970,-21.101156,1.964660,-16.300379,-8.601916,4.498190,-1.829921,-6.888492,6.807146
...,...,...,...,...,...,...,...,...,...,...,...,...
2908,-853.85474,30.689743,24.807337,27.891415,24.391132,1.348291,10.089901,27.455496,7.106500,0.076568,20.415947,12.648972
2909,-515.60620,117.292404,-28.858470,-0.166154,-22.809288,10.571767,-7.357612,1.925440,9.165670,4.499895,-5.087014,7.028556
2910,-483.52502,122.156690,-39.812244,-3.797422,-9.360095,-2.103504,1.424180,-3.274032,-0.363364,17.463123,-7.233619,0.365348
2911,-838.04193,21.284880,18.860140,30.818695,22.122593,1.856854,12.400106,22.434920,5.217744,-1.222926,18.654396,13.201369


In [ ]:
y = one_hot_encoded
X = (X-X.min())/(X.max()-X.min())
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.2)

In [ ]:
X_train.shape

(2330, 12)

In [ ]:
type(X)

pandas.core.frame.DataFrame

In [ ]:
X_train.shape


(2330, 12)

## **MLP classifier**

**MFCC**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/MFCC2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
# You can adjust the hidden_layer_sizes and other hyperparameters as needed

# Fit the MLP model
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.937070938215103
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.97      0.93      0.95       189
   Bass Tuba       0.94      0.91      0.93        35
     Bassoon       0.96      0.92      0.94        49
    Clarinet       0.90      0.76      0.83        34
  Contrabass       0.97      1.00      0.98        95
       Flute       0.92      0.85      0.88        39
        Horn       0.97      0.92      0.95        39
        Oboe       0.85      0.97      0.90        29
    Sax_Alto       0.82      0.82      0.82        28
    Trombone       0.92      0.97      0.95        36
     Trumpet       0.97      0.97      0.97        32
       Viola       0.85      0.97      0.90        86
      Violin       0.98      0.92      0.95        89
 Violoncello       0.95      1.00      0.97        94

    accuracy                           0.94       874
   macro avg       0.93      0.92      0.92       874
weighted avg       0.94      

**chroma**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/chroma_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
# You can adjust the hidden_layer_sizes and other hyperparameters as needed

# Fit the MLP model
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.5995423340961098
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.86      0.94      0.90       189
   Bass Tuba       0.35      0.34      0.35        35
     Bassoon       0.43      0.39      0.41        49
    Clarinet       0.17      0.09      0.12        34
  Contrabass       0.72      0.85      0.78        95
       Flute       0.26      0.23      0.24        39
        Horn       0.47      0.36      0.41        39
        Oboe       0.42      0.59      0.49        29
    Sax_Alto       0.05      0.04      0.04        28
    Trombone       0.31      0.22      0.26        36
     Trumpet       0.32      0.19      0.24        32
       Viola       0.57      0.76      0.65        86
      Violin       0.66      0.67      0.67        89
 Violoncello       0.62      0.54      0.58        94

    accuracy                           0.60       874
   macro avg       0.44      0.44      0.44       874
weighted avg       0.57     

wavelet **transform**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/wavelet_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
# You can adjust the hidden_layer_sizes and other hyperparameters as needed

# Fit the MLP model
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.2195540308747856
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.22      1.00      0.36       128
   Bass Tuba       0.00      0.00      0.00        23
     Bassoon       0.00      0.00      0.00        34
    Clarinet       0.00      0.00      0.00        21
  Contrabass       0.00      0.00      0.00        64
       Flute       0.00      0.00      0.00        27
        Horn       0.00      0.00      0.00        31
        Oboe       0.00      0.00      0.00        20
    Sax_Alto       0.00      0.00      0.00        12
    Trombone       0.00      0.00      0.00        26
     Trumpet       0.00      0.00      0.00        22
       Viola       0.00      0.00      0.00        51
      Violin       0.00      0.00      0.00        57
 Violoncello       0.00      0.00      0.00        67

    accuracy                           0.22       583
   macro avg       0.02      0.07      0.03       583
weighted avg       0.05     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


spectral **contrast**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/spectral_contrast_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
# You can adjust the hidden_layer_sizes and other hyperparameters as needed

# Fit the MLP model
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.6705559368565546
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.81      0.84      0.83       334
   Bass Tuba       0.59      0.50      0.54        54
     Bassoon       0.66      0.64      0.65        72
    Clarinet       0.59      0.72      0.65        61
  Contrabass       0.77      0.86      0.82       162
       Flute       0.66      0.74      0.70        58
        Horn       0.45      0.53      0.48        59
        Oboe       0.61      0.70      0.65        53
    Sax_Alto       0.42      0.10      0.17        48
    Trombone       0.46      0.46      0.46        54
     Trumpet       0.65      0.28      0.39        53
       Viola       0.58      0.46      0.52       155
      Violin       0.65      0.69      0.67       136
 Violoncello       0.63      0.73      0.68       158

    accuracy                           0.67      1457
   macro avg       0.61      0.59      0.59      1457
weighted avg       0.66     

**mixed**

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/mixed_feature2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["38"]).values
y = dataset["38"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
# You can adjust the hidden_layer_sizes and other hyperparameters as needed

# Fit the MLP model
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.9835390946502057
Classification Report:
              precision    recall  f1-score   support

   Accordion       1.00      0.99      1.00       159
   Bass Tuba       0.88      1.00      0.94        29
     Bassoon       0.97      0.95      0.96        40
    Clarinet       1.00      0.96      0.98        26
  Contrabass       0.99      1.00      0.99        74
       Flute       1.00      1.00      1.00        34
        Horn       1.00      0.83      0.91        35
        Oboe       1.00      1.00      1.00        26
    Sax_Alto       1.00      1.00      1.00        20
    Trombone       0.91      0.97      0.94        32
     Trumpet       1.00      1.00      1.00        27
       Viola       0.99      1.00      0.99        69
      Violin       1.00      0.99      0.99        76
 Violoncello       0.98      1.00      0.99        82

    accuracy                           0.98       729
   macro avg       0.98      0.98      0.98       729
weighted avg       0.98     

## KNN

**MFCC**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/MFCC2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=12)  # You can adjust the number of neighbors (k) here

# No training is needed for KNN

# Fit the KNN model (not a traditional training, just organizing the data)
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.8375286041189931
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.87      0.92      0.89       189
   Bass Tuba       0.80      0.91      0.85        35
     Bassoon       0.84      0.73      0.78        49
    Clarinet       0.67      0.47      0.55        34
  Contrabass       0.93      0.99      0.96        95
       Flute       0.79      0.59      0.68        39
        Horn       0.67      0.51      0.58        39
        Oboe       0.68      0.66      0.67        29
    Sax_Alto       0.75      0.43      0.55        28
    Trombone       0.68      0.72      0.70        36
     Trumpet       0.93      0.81      0.87        32
       Viola       0.78      0.94      0.85        86
      Violin       0.87      0.96      0.91        89
 Violoncello       0.93      0.95      0.94        94

    accuracy                           0.84       874
   macro avg       0.80      0.76      0.77       874
weighted avg       0.83     

**CHROMA**

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/chroma_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) here

# No training is needed for KNN

# Fit the KNN model (not a traditional training, just organizing the data)
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.5205949656750573
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.70      0.93      0.80       189
   Bass Tuba       0.23      0.23      0.23        35
     Bassoon       0.43      0.41      0.42        49
    Clarinet       0.08      0.06      0.07        34
  Contrabass       0.65      0.83      0.73        95
       Flute       0.28      0.18      0.22        39
        Horn       0.18      0.10      0.13        39
        Oboe       0.33      0.31      0.32        29
    Sax_Alto       0.00      0.00      0.00        28
    Trombone       0.09      0.03      0.04        36
     Trumpet       0.00      0.00      0.00        32
       Viola       0.47      0.66      0.55        86
      Violin       0.49      0.57      0.53        89
 Violoncello       0.57      0.45      0.50        94

    accuracy                           0.52       874
   macro avg       0.32      0.34      0.32       874
weighted avg       0.45     

**wavelet transform**

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/wavelet_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) here

# No training is needed for KNN

# Fit the KNN model (not a traditional training, just organizing the data)
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.2830188679245283
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.44      0.79      0.57       128
   Bass Tuba       0.19      0.13      0.15        23
     Bassoon       0.00      0.00      0.00        34
    Clarinet       0.05      0.10      0.07        21
  Contrabass       0.33      0.23      0.28        64
       Flute       0.00      0.00      0.00        27
        Horn       0.10      0.06      0.08        31
        Oboe       0.00      0.00      0.00        20
    Sax_Alto       0.12      0.08      0.10        12
    Trombone       0.17      0.08      0.11        26
     Trumpet       0.40      0.09      0.15        22
       Viola       0.18      0.27      0.22        51
      Violin       0.18      0.19      0.18        57
 Violoncello       0.27      0.18      0.22        67

    accuracy                           0.28       583
   macro avg       0.17      0.16      0.15       583
weighted avg       0.24     

spectral contrast

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/spectral_contrast_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) here

# No training is needed for KNN

# Fit the KNN model (not a traditional training, just organizing the data)
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7356979405034325
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.79      0.90      0.84       189
   Bass Tuba       0.65      0.63      0.64        35
     Bassoon       0.76      0.57      0.65        49
    Clarinet       0.80      0.59      0.68        34
  Contrabass       0.79      0.94      0.86        95
       Flute       0.81      0.77      0.79        39
        Horn       0.60      0.64      0.62        39
        Oboe       0.76      0.86      0.81        29
    Sax_Alto       0.65      0.54      0.59        28
    Trombone       0.62      0.36      0.46        36
     Trumpet       0.65      0.62      0.63        32
       Viola       0.63      0.69      0.66        86
      Violin       0.79      0.65      0.72        89
 Violoncello       0.71      0.73      0.72        94

    accuracy                           0.74       874
   macro avg       0.71      0.68      0.69       874
weighted avg       0.73     

MIXED

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/mixed_feature2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["38"]).values
y = dataset["38"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) here

# No training is needed for KNN

# Fit the KNN model (not a traditional training, just organizing the data)
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.9919908466819222
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.99      0.99      0.99       189
   Bass Tuba       1.00      1.00      1.00        35
     Bassoon       1.00      1.00      1.00        49
    Clarinet       1.00      1.00      1.00        34
  Contrabass       1.00      1.00      1.00        95
       Flute       1.00      0.92      0.96        39
        Horn       1.00      0.95      0.97        39
        Oboe       0.91      1.00      0.95        29
    Sax_Alto       0.96      0.96      0.96        28
    Trombone       0.95      1.00      0.97        36
     Trumpet       1.00      1.00      1.00        32
       Viola       1.00      1.00      1.00        86
      Violin       1.00      1.00      1.00        89
 Violoncello       1.00      1.00      1.00        94

    accuracy                           0.99       874
   macro avg       0.99      0.99      0.99       874
weighted avg       0.99     

# **Random Forest**

Using **MFCC**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/MFCC2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.9473684210526315
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.94      0.97      0.96       189
   Bass Tuba       0.92      0.97      0.94        35
     Bassoon       0.98      0.96      0.97        49
    Clarinet       0.92      0.71      0.80        34
  Contrabass       0.99      1.00      0.99        95
       Flute       0.97      0.87      0.92        39
        Horn       0.94      0.87      0.91        39
        Oboe       0.88      0.97      0.92        29
    Sax_Alto       0.85      0.82      0.84        28
    Trombone       0.92      0.92      0.92        36
     Trumpet       0.97      0.97      0.97        32
       Viola       0.94      0.97      0.95        86
      Violin       0.92      0.97      0.95        89
 Violoncello       0.99      0.98      0.98        94

    accuracy                           0.95       874
   macro avg       0.94      0.92      0.93       874
weighted avg       0.95     

**CHROMA**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/chroma_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.6132723112128147
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.78      0.95      0.86       189
   Bass Tuba       0.21      0.09      0.12        35
     Bassoon       0.67      0.57      0.62        49
    Clarinet       0.15      0.15      0.15        34
  Contrabass       0.79      0.82      0.80        95
       Flute       0.36      0.31      0.33        39
        Horn       0.57      0.33      0.42        39
        Oboe       0.41      0.52      0.45        29
    Sax_Alto       0.12      0.11      0.11        28
    Trombone       0.25      0.08      0.12        36
     Trumpet       0.20      0.06      0.10        32
       Viola       0.58      0.79      0.67        86
      Violin       0.65      0.71      0.68        89
 Violoncello       0.64      0.67      0.66        94

    accuracy                           0.61       874
   macro avg       0.45      0.44      0.43       874
weighted avg       0.57     

**WAVELET Transform**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/wavelet_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.2195540308747856
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.22      1.00      0.36       128
   Bass Tuba       0.00      0.00      0.00        23
     Bassoon       0.00      0.00      0.00        34
    Clarinet       0.00      0.00      0.00        21
  Contrabass       0.00      0.00      0.00        64
       Flute       0.00      0.00      0.00        27
        Horn       0.00      0.00      0.00        31
        Oboe       0.00      0.00      0.00        20
    Sax_Alto       0.00      0.00      0.00        12
    Trombone       0.00      0.00      0.00        26
     Trumpet       0.00      0.00      0.00        22
       Viola       0.00      0.00      0.00        51
      Violin       0.00      0.00      0.00        57
 Violoncello       0.00      0.00      0.00        67

    accuracy                           0.22       583
   macro avg       0.02      0.07      0.03       583
weighted avg       0.05     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**spectral contrast**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/spectral_contrast_features2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["label_list"]).values
y = dataset["label_list"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7860411899313501
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.80      0.94      0.87       189
   Bass Tuba       0.83      0.69      0.75        35
     Bassoon       0.69      0.49      0.57        49
    Clarinet       0.93      0.74      0.82        34
  Contrabass       0.83      0.95      0.89        95
       Flute       0.82      0.85      0.84        39
        Horn       0.68      0.59      0.63        39
        Oboe       0.90      0.90      0.90        29
    Sax_Alto       0.59      0.68      0.63        28
    Trombone       0.54      0.36      0.43        36
     Trumpet       0.68      0.66      0.67        32
       Viola       0.75      0.77      0.76        86
      Violin       0.84      0.80      0.82        89
 Violoncello       0.82      0.79      0.80        94

    accuracy                           0.79       874
   macro avg       0.76      0.73      0.74       874
weighted avg       0.78     

**mixed**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset from the CSV file
dataset_path = "/content/drive/MyDrive/mini project/mixed_feature2.csv"
dataset = pd.read_csv(dataset_path)

# Create feature matrix (X) and target labels (y)
X = dataset.drop(columns=["38"]).values
y = dataset["38"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.982837528604119
Classification Report:
              precision    recall  f1-score   support

   Accordion       0.98      1.00      0.99       189
   Bass Tuba       0.95      1.00      0.97        35
     Bassoon       1.00      1.00      1.00        49
    Clarinet       1.00      0.94      0.97        34
  Contrabass       0.99      1.00      0.99        95
       Flute       0.97      0.95      0.96        39
        Horn       1.00      0.97      0.99        39
        Oboe       1.00      1.00      1.00        29
    Sax_Alto       0.93      0.96      0.95        28
    Trombone       0.97      0.97      0.97        36
     Trumpet       1.00      0.97      0.98        32
       Viola       0.99      0.95      0.97        86
      Violin       0.98      0.97      0.97        89
 Violoncello       0.99      1.00      0.99        94

    accuracy                           0.98       874
   macro avg       0.98      0.98      0.98       874
weighted avg       0.98      

In [ ]:
dataset

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
0,0,-659.21630,18.278786,-12.107536,44.013153,-13.154768,5.019222,-5.396541,-3.083084,-9.133388,...,21.314483,16.162295,15.162575,25.197684,22.266555,26.871772,26.033675,21.314483,16.162295,Violin
1,1,-533.87380,33.250694,-24.455145,43.151917,-25.093418,-10.435495,-19.112411,-15.169186,-7.704080,...,20.374345,15.423609,15.484341,25.202887,24.579884,26.081575,25.067514,20.374345,15.423609,Violin
2,2,-468.43643,11.820848,-19.447367,33.744300,-24.006926,-10.317425,-12.795439,-13.549147,-10.402909,...,21.514069,17.738277,16.281310,26.613672,24.604883,27.214358,26.568307,21.514069,17.738277,Violin
3,3,-594.96625,10.194246,-8.782218,40.609173,-10.718865,0.291726,-3.624802,-6.048359,-7.304923,...,22.624560,18.797797,15.776857,25.668242,23.883489,27.364402,28.478042,22.624560,18.797797,Violin
4,4,-635.07110,58.709858,-32.574550,46.426970,-21.101156,1.964660,-16.300379,-8.601916,4.498190,...,18.141995,13.921893,23.834682,24.589448,26.844933,20.721276,21.594385,18.141995,13.921893,Violin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,2908,-853.85474,30.689743,24.807337,27.891415,24.391132,1.348291,10.089901,27.455496,7.106500,...,23.438054,16.812360,20.090048,10.443753,12.450319,30.848957,21.604394,23.438054,16.812360,Accordion
2909,2909,-515.60620,117.292404,-28.858470,-0.166154,-22.809288,10.571767,-7.357612,1.925440,9.165670,...,18.572611,15.719823,24.028890,17.766160,20.336524,20.065962,18.779595,18.572611,15.719823,Accordion
2910,2910,-483.52502,122.156690,-39.812244,-3.797422,-9.360095,-2.103504,1.424180,-3.274032,-0.363364,...,18.595865,16.487032,23.615017,17.293229,20.440290,20.330123,22.441011,18.595865,16.487032,Accordion
2911,2911,-838.04193,21.284880,18.860140,30.818695,22.122593,1.856854,12.400106,22.434920,5.217744,...,27.156039,21.744066,18.793144,10.396214,12.444192,32.240963,25.115234,27.156039,21.744066,Accordion


# CNN

**MfCC**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/MFCC2.csv')

# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# If the data is in 2D format, reshape it to 3D (num_samples, num_timesteps, num_features)
# Example: x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:

# Create the CNN model
model = keras.Sequential([
    layers.Reshape((12, 1), input_shape=(12,)),  # Reshape input for CNN
    layers.Conv1D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

# Train the model
batch_size = 16
epochs = 20

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
148/148 [==============================] - 2s 5ms/step - loss: 2.2024 - accuracy: 0.2761 - val_loss: 1.6255 - val_accuracy: 0.4525
Epoch 2/20
148/148 [==============================] - 1s 3ms/step - loss: 1.2707 - accuracy: 0.5700 - val_loss: 0.9897 - val_accuracy: 0.7338
Epoch 3/20
148/148 [==============================] - 0s 3ms/step - loss: 0.9075 - accuracy: 0.6900 - val_loss: 0.7552 - val_accuracy: 0.7490
Epoch 4/20
148/148 [==============================] - 1s 6ms/step - loss: 0.7124 - accuracy: 0.7549 - val_loss: 0.6305 - val_accuracy: 0.7909
Epoch 5/20
148/148 [==============================] - 1s 4ms/step - loss: 0.5945 - accuracy: 0.7990 - val_loss: 0.5057 - val_accuracy: 0.8251
Epoch 6/20
148/148 [==============================] - 0s 3ms/step - loss: 0.5249 - accuracy: 0.8240 - val_loss: 0.4494 - val_accuracy: 0.8517
Epoch 7/20
148/148 [==============================] - 0s 3ms/step - loss: 0.4563 - accuracy: 0.8486 - val_loss: 0.3785 - val_accuracy: 0.8897
Epoch 

In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

10/10 [==============================] - 0s 3ms/step - loss: 0.1503 - accuracy: 0.9521
Test accuracy: 0.9520547986030579


In [ ]:
# Specify the path where you want to save the model
model_path = '/content/drive/MyDrive/CNN2_saved_model.h5'

# Save the model to the specified path
model.save(model_path)
print("Model saved successfully at:", model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully at: /content/drive/MyDrive/CNN2_saved_model.h5


In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

10/10 [==============================] - 0s 3ms/step
Macro Avg Precision: 0.9309917463488893
Macro Avg Recall: 0.9336618907034685
Macro Avg F1-score: 0.930952957635211


**Chroma**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/chroma_features2.csv')

# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# If the data is in 2D format, reshape it to 3D (num_samples, num_timesteps, num_features)
# Example: x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Create the CNN model
model = keras.Sequential([
    layers.Reshape((12, 1), input_shape=(12,)),  # Reshape input for CNN
    layers.Conv1D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




# Train the model
batch_size = 16
epochs = 20


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
148/148 [==============================] - 2s 4ms/step - loss: 2.2833 - accuracy: 0.2803 - val_loss: 2.0249 - val_accuracy: 0.3536
Epoch 2/20
148/148 [==============================] - 0s 3ms/step - loss: 1.9942 - accuracy: 0.3783 - val_loss: 1.8396 - val_accuracy: 0.4259
Epoch 3/20
148/148 [==============================] - 0s 3ms/step - loss: 1.8351 - accuracy: 0.4135 - val_loss: 1.7289 - val_accuracy: 0.4373
Epoch 4/20
148/148 [==============================] - 0s 3ms/step - loss: 1.6908 - accuracy: 0.4550 - val_loss: 1.6223 - val_accuracy: 0.4487
Epoch 5/20
148/148 [==============================] - 0s 3ms/step - loss: 1.5859 - accuracy: 0.4843 - val_loss: 1.5379 - val_accuracy: 0.4867
Epoch 6/20
148/148 [==============================] - 0s 3ms/step - loss: 1.5104 - accuracy: 0.4924 - val_loss: 1.4266 - val_accuracy: 0.4905
Epoch 7/20
148/148 [==============================] - 0s 3ms/step - loss: 1.4479 - accuracy: 0.4996 - val_loss: 1.3788 - val_accuracy: 0.5019
Epoch 

In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

10/10 [==============================] - 0s 3ms/step
Macro Avg Precision: 0.45803229470565976
Macro Avg Recall: 0.4769862261781866
Macro Avg F1-score: 0.447248771430309


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


wavelet transform

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/wavelet_features2.csv')

# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# If the data is in 2D format, reshape it to 3D (num_samples, num_timesteps, num_features)
# Example: x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Create the CNN model
model = keras.Sequential([
    layers.Reshape((10, 1), input_shape=(10,)),  # Reshape input for CNN
    layers.Conv1D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




# Train the model
batch_size = 16
epochs = 20


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
132/132 [==============================] - 2s 7ms/step - loss: 2.4548 - accuracy: 0.2413 - val_loss: 2.3466 - val_accuracy: 0.2661
Epoch 2/20
132/132 [==============================] - 0s 3ms/step - loss: 2.3767 - accuracy: 0.2604 - val_loss: 2.3050 - val_accuracy: 0.2876
Epoch 3/20
132/132 [==============================] - 0s 3ms/step - loss: 2.3286 - accuracy: 0.2661 - val_loss: 2.2745 - val_accuracy: 0.2918
Epoch 4/20
132/132 [==============================] - 0s 3ms/step - loss: 2.3048 - accuracy: 0.2694 - val_loss: 2.2895 - val_accuracy: 0.2918
Epoch 5/20
132/132 [==============================] - 0s 3ms/step - loss: 2.2822 - accuracy: 0.2728 - val_loss: 2.2014 - val_accuracy: 0.2918
Epoch 6/20
132/132 [==============================] - 0s 3ms/step - loss: 2.2454 - accuracy: 0.2756 - val_loss: 2.1746 - val_accuracy: 0.2961
Epoch 7/20
132/132 [==============================] - 0s 3ms/step - loss: 2.2074 - accuracy: 0.2842 - val_loss: 2.1156 - val_accuracy: 0.3133
Epoch 

spactral contrast

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/spectral_contrast_features2.csv')

# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# If the data is in 2D format, reshape it to 3D (num_samples, num_timesteps, num_features)
# Example: x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Create the CNN model
model = keras.Sequential([
    layers.Reshape((7, 1), input_shape=(7,)),  # Reshape input for CNN
    layers.Conv1D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




# Train the model
batch_size = 16
epochs = 20


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
148/148 [==============================] - 2s 6ms/step - loss: 2.4415 - accuracy: 0.2256 - val_loss: 2.2973 - val_accuracy: 0.2471
Epoch 2/20
148/148 [==============================] - 1s 4ms/step - loss: 2.2175 - accuracy: 0.2850 - val_loss: 2.0129 - val_accuracy: 0.3802
Epoch 3/20
148/148 [==============================] - 1s 4ms/step - loss: 1.9980 - accuracy: 0.3363 - val_loss: 1.7946 - val_accuracy: 0.3878
Epoch 4/20
148/148 [==============================] - 1s 4ms/step - loss: 1.8467 - accuracy: 0.3673 - val_loss: 1.6651 - val_accuracy: 0.4259
Epoch 5/20
148/148 [==============================] - 1s 4ms/step - loss: 1.7615 - accuracy: 0.3885 - val_loss: 1.6054 - val_accuracy: 0.4715
Epoch 6/20
148/148 [==============================] - 0s 3ms/step - loss: 1.6917 - accuracy: 0.4037 - val_loss: 1.5403 - val_accuracy: 0.4563
Epoch 7/20
148/148 [==============================] - 0s 3ms/step - loss: 1.6437 - accuracy: 0.4088 - val_loss: 1.5276 - val_accuracy: 0.4601
Epoch 

In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

10/10 [==============================] - 0s 2ms/step
Macro Avg Precision: 0.564622437603804
Macro Avg Recall: 0.509363274586528
Macro Avg F1-score: 0.5111929595514983


**mixed**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/mixed_feature2.csv')

# Extract features and labels
features = data.drop('38', axis=1).values
labels = data['38'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# If the data is in 2D format, reshape it to 3D (num_samples, num_timesteps, num_features)
# Example: x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Create the CNN model
model = keras.Sequential([
    layers.Reshape((39, 1), input_shape=(39,)),  # Reshape input for CNN
    layers.Conv1D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




# Train the model
batch_size = 16
epochs = 20


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

Epoch 1/20
148/148 [==============================] - 2s 7ms/step - loss: 2.2294 - accuracy: 0.2820 - val_loss: 1.9023 - val_accuracy: 0.3270
Epoch 2/20
148/148 [==============================] - 1s 6ms/step - loss: 1.7954 - accuracy: 0.3261 - val_loss: 1.6455 - val_accuracy: 0.3840
Epoch 3/20
148/148 [==============================] - 1s 6ms/step - loss: 1.5028 - accuracy: 0.4436 - val_loss: 1.2793 - val_accuracy: 0.4601
Epoch 4/20
148/148 [==============================] - 1s 8ms/step - loss: 1.0853 - accuracy: 0.6289 - val_loss: 0.8587 - val_accuracy: 0.7605
Epoch 5/20
148/148 [==============================] - 1s 7ms/step - loss: 0.7480 - accuracy: 0.7447 - val_loss: 0.6638 - val_accuracy: 0.7681
Epoch 6/20
148/148 [==============================] - 1s 6ms/step - loss: 0.5867 - accuracy: 0.7956 - val_loss: 0.5354 - val_accuracy: 0.7909
Epoch 7/20
148/148 [==============================] - 1s 5ms/step - loss: 0.4709 - accuracy: 0.8338 - val_loss: 0.4241 - val_accuracy: 0.8555
Epoch 

# **RNN**

MFCC

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/MFCC2.csv')

In [ ]:
# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# Reshape the data to 3D format (num_samples, num_timesteps, num_features)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
x_train.shape

(2621, 12, 1)

In [ ]:
# Create the RNN model using LSTM
model = keras.Sequential([
    layers.LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 20

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
148/148 [==============================] - 9s 22ms/step - loss: 2.4170 - accuracy: 0.2379 - val_loss: 2.3206 - val_accuracy: 0.2700
Epoch 2/20
148/148 [==============================] - 2s 14ms/step - loss: 2.1271 - accuracy: 0.2947 - val_loss: 1.8543 - val_accuracy: 0.3384
Epoch 3/20
148/148 [==============================] - 2s 15ms/step - loss: 1.8462 - accuracy: 0.3494 - val_loss: 1.6673 - val_accuracy: 0.3878
Epoch 4/20
148/148 [==============================] - 3s 23ms/step - loss: 1.5873 - accuracy: 0.4419 - val_loss: 1.5192 - val_accuracy: 0.4373
Epoch 5/20
148/148 [==============================] - 3s 18ms/step - loss: 1.3517 - accuracy: 0.5030 - val_loss: 1.1500 - val_accuracy: 0.5817
Epoch 6/20
148/148 [==============================] - 2s 14ms/step - loss: 1.2628 - accuracy: 0.5288 - val_loss: 1.0972 - val_accuracy: 0.6008
Epoch 7/20
148/148 [==============================] - 2s 16ms/step - loss: 1.1802 - accuracy: 0.5602 - val_loss: 1.0322 - val_accuracy: 0.6388

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

10/10 [==============================] - 0s 6ms/step - loss: 0.7073 - accuracy: 0.7158
Test accuracy: 0.715753436088562


In [ ]:
# Specify the path where you want to save the model
model_path = '/content/drive/MyDrive/Rnn2_saved_model.h5'

# Save the model to the specified path
model.save(model_path)

print("Model saved successfully at:", model_path)

Model saved successfully at: /content/drive/MyDrive/Rnn2_saved_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

10/10 [==============================] - 1s 8ms/step
Macro Avg Precision: 0.5838457504832842
Macro Avg Recall: 0.6153893710182121
Macro Avg F1-score: 0.5919985361271454


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


chroma

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers


# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/chroma_features2.csv')


# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# Reshape the data to 3D format (num_samples, num_timesteps, num_features)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Create the RNN model using LSTM
model = keras.Sequential([
    layers.LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 20

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
132/132 [==============================] - 8s 23ms/step - loss: 2.3328 - accuracy: 0.2537 - val_loss: 2.1642 - val_accuracy: 0.3176
Epoch 2/20
132/132 [==============================] - 2s 15ms/step - loss: 2.1357 - accuracy: 0.3085 - val_loss: 2.1045 - val_accuracy: 0.3176
Epoch 3/20
132/132 [==============================] - 2s 18ms/step - loss: 2.1008 - accuracy: 0.3181 - val_loss: 2.0233 - val_accuracy: 0.3476
Epoch 4/20
132/132 [==============================] - 3s 23ms/step - loss: 2.0400 - accuracy: 0.3357 - val_loss: 1.9600 - val_accuracy: 0.3734
Epoch 5/20
132/132 [==============================] - 2s 17ms/step - loss: 1.9927 - accuracy: 0.3600 - val_loss: 1.9486 - val_accuracy: 0.3648
Epoch 6/20
132/132 [==============================] - 2s 15ms/step - loss: 1.9408 - accuracy: 0.3801 - val_loss: 1.9027 - val_accuracy: 0.3777
Epoch 7/20
132/132 [==============================] - 2s 15ms/step - loss: 1.8711 - accuracy: 0.3882 - val_loss: 1.8199 - val_accuracy: 0.4077

In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

19/19 [==============================] - 1s 5ms/step
Macro Avg Precision: 0.3536602127345768
Macro Avg Recall: 0.3726314597122399
Macro Avg F1-score: 0.35450945613269846


wavelet transform

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers


# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/wavelet_features2.csv')


# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# Reshape the data to 3D format (num_samples, num_timesteps, num_features)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Create the RNN model using LSTM
model = keras.Sequential([
    layers.LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 20

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
132/132 [==============================] - 9s 30ms/step - loss: 2.4585 - accuracy: 0.2356 - val_loss: 2.3822 - val_accuracy: 0.2489
Epoch 2/20
132/132 [==============================] - 2s 15ms/step - loss: 2.4238 - accuracy: 0.2399 - val_loss: 2.3928 - val_accuracy: 0.2489
Epoch 3/20
132/132 [==============================] - 2s 15ms/step - loss: 2.4229 - accuracy: 0.2399 - val_loss: 2.3831 - val_accuracy: 0.2489
Epoch 4/20
132/132 [==============================] - 2s 15ms/step - loss: 2.4242 - accuracy: 0.2399 - val_loss: 2.3780 - val_accuracy: 0.2489
Epoch 5/20
132/132 [==============================] - 2s 14ms/step - loss: 2.4241 - accuracy: 0.2399 - val_loss: 2.3779 - val_accuracy: 0.2489
Epoch 6/20
132/132 [==============================] - 2s 16ms/step - loss: 2.4200 - accuracy: 0.2399 - val_loss: 2.3782 - val_accuracy: 0.2489
Epoch 7/20
132/132 [==============================] - 3s 22ms/step - loss: 2.4194 - accuracy: 0.2399 - val_loss: 2.3783 - val_accuracy: 0.2489

spectral contrast

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers


# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/spectral_contrast_features2.csv')


# Extract features and labels
features = data.drop('label_list', axis=1).values
labels = data['label_list'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# Reshape the data to 3D format (num_samples, num_timesteps, num_features)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Create the RNN model using LSTM
model = keras.Sequential([
    layers.LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 20

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
148/148 [==============================] - 8s 17ms/step - loss: 2.4653 - accuracy: 0.2303 - val_loss: 2.3994 - val_accuracy: 0.2471
Epoch 2/20
148/148 [==============================] - 2s 11ms/step - loss: 2.3783 - accuracy: 0.2345 - val_loss: 2.2391 - val_accuracy: 0.2586
Epoch 3/20
148/148 [==============================] - 2s 10ms/step - loss: 2.2286 - accuracy: 0.2439 - val_loss: 2.1269 - val_accuracy: 0.2662
Epoch 4/20
148/148 [==============================] - 2s 11ms/step - loss: 2.0909 - accuracy: 0.2930 - val_loss: 1.9213 - val_accuracy: 0.3650
Epoch 5/20
148/148 [==============================] - 2s 12ms/step - loss: 1.9568 - accuracy: 0.3193 - val_loss: 1.9285 - val_accuracy: 0.2890
Epoch 6/20
148/148 [==============================] - 3s 18ms/step - loss: 1.8315 - accuracy: 0.3528 - val_loss: 1.7941 - val_accuracy: 0.3574
Epoch 7/20
148/148 [==============================] - 2s 15ms/step - loss: 1.7846 - accuracy: 0.3664 - val_loss: 1.6388 - val_accuracy: 0.4297

In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

10/10 [==============================] - 1s 4ms/step
Macro Avg Precision: 0.5018286801900247
Macro Avg Recall: 0.3899092237354184
Macro Avg F1-score: 0.3860961811928938


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**mixed**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers


# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/mini project/mixed_feature2.csv')


# Extract features and labels
features = data.drop('38', axis=1).values
labels = data['38'].values

# Convert labels to integers using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# Normalize the features to range [0, 1]
x_train = x_train.astype('float32') / np.max(features)
x_test = x_test.astype('float32') / np.max(features)

# Reshape the data to 3D format (num_samples, num_timesteps, num_features)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Create the RNN model using LSTM
model = keras.Sequential([
    layers.LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True),
    layers.LSTM(64),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 16
epochs = 20

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

Epoch 1/20
148/148 [==============================] - 13s 45ms/step - loss: 2.4564 - accuracy: 0.2320 - val_loss: 2.3973 - val_accuracy: 0.2471
Epoch 2/20
148/148 [==============================] - 7s 50ms/step - loss: 2.4273 - accuracy: 0.2362 - val_loss: 2.4054 - val_accuracy: 0.2471
Epoch 3/20
148/148 [==============================] - 6s 38ms/step - loss: 2.4278 - accuracy: 0.2362 - val_loss: 2.3897 - val_accuracy: 0.2471
Epoch 4/20
148/148 [==============================] - 7s 49ms/step - loss: 2.3520 - accuracy: 0.2646 - val_loss: 2.2364 - val_accuracy: 0.3080
Epoch 5/20
148/148 [==============================] - 6s 40ms/step - loss: 2.2457 - accuracy: 0.2918 - val_loss: 2.2568 - val_accuracy: 0.2700
Epoch 6/20
148/148 [==============================] - 7s 50ms/step - loss: 2.2385 - accuracy: 0.2854 - val_loss: 2.2215 - val_accuracy: 0.3042
Epoch 7/20
148/148 [==============================] - 6s 38ms/step - loss: 2.2319 - accuracy: 0.2557 - val_loss: 2.0445 - val_accuracy: 0.254

In [ ]:
# Make predictions on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate precision, recall, and F1-score with 'macro' averaging
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1 = f1_score(y_true, y_pred_classes, average='macro')

print('Macro Avg Precision:', precision)
print('Macro Avg Recall:', recall)
print('Macro Avg F1-score:', f1)

10/10 [==============================] - 1s 16ms/step
Macro Avg Precision: 0.04734051936235598
Macro Avg Recall: 0.14285714285714285
Macro Avg F1-score: 0.07099884708101876


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
